In [ ]:
import pandas as pd
from bokeh.plotting import figure, show

usa_census = pd.read_csv(
    './data/USA/DECENNIALDDHCA2020.T01001_2024-12-15T154541/DECENNIALDDHCA2020.T01001-Data.csv'
)

usa_census.drop(index=usa_census.index[0], axis=0, inplace=True)

usa_census.drop(usa_census.columns[5], axis=1, inplace=True)

usa_census['in_any_combination'] = usa_census['POPGROUP_LABEL'].match('in any combination')

display(usa_census.groupby(by='POPGROUP_LABEL')['POPGROUP_LABEL'])

# p = figure(title="Hello Plot!", x_axis_label='blah', y_axis_label='foo')
